In [11]:
from kerchunk.hdf import SingleHdf5ToZarr 
from kerchunk.combine import MultiZarrToZarr
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import s3fs
import datetime as dt
import logging
import fsspec
import ujson
from tqdm import tqdm
from glob import glob

In [12]:
from fsspec.registry import known_implementations
known_implementations.keys()

dict_keys(['file', 'local', 'memory', 'dropbox', 'http', 'https', 'zip', 'tar', 'gcs', 'gs', 'gdrive', 'sftp', 'ssh', 'ftp', 'hdfs', 'arrow_hdfs', 'webhdfs', 's3', 's3a', 'wandb', 'oci', 'ocilake', 'asynclocal', 'adl', 'abfs', 'az', 'cached', 'blockcache', 'filecache', 'simplecache', 'dask', 'dbfs', 'github', 'git', 'smb', 'jupyter', 'jlab', 'libarchive', 'reference', 'generic', 'oss', 'webdav', 'dvc', 'hf', 'root', 'dir', 'box', 'lakefs'])

In [13]:
fs = fsspec.filesystem('s3', anon=True)

In [20]:
urls = ['s3://' + f for f in fs.glob("s3://noaa-goes16/ABI-L1b-RadC/2000/001/*/*.nc")]
urls

['s3://noaa-goes16/ABI-L1b-RadC/2000/001/12/OR_ABI-L1b-RadC-M3C01_G16_s20000011200000_e20000011200000_c20170671748180.nc',
 's3://noaa-goes16/ABI-L1b-RadC/2000/001/12/OR_ABI-L1b-RadC-M3C01_G16_s20000011200000_e20000011200000_c20170691603180.nc',
 's3://noaa-goes16/ABI-L1b-RadC/2000/001/12/OR_ABI-L1b-RadC-M3C01_G16_s20000011200000_e20000011200000_c20170751219598.nc',
 's3://noaa-goes16/ABI-L1b-RadC/2000/001/12/OR_ABI-L1b-RadC-M3C01_G16_s20000011200000_e20000011200000_c20170752149454.nc',
 's3://noaa-goes16/ABI-L1b-RadC/2000/001/12/OR_ABI-L1b-RadC-M3C01_G16_s20000011200000_e20000011200000_c20170752204183.nc',
 's3://noaa-goes16/ABI-L1b-RadC/2000/001/12/OR_ABI-L1b-RadC-M3C01_G16_s20000011200000_e20000011200000_c20170752234173.nc',
 's3://noaa-goes16/ABI-L1b-RadC/2000/001/12/OR_ABI-L1b-RadC-M3C01_G16_s20000011200000_e20000011200000_c20170901216521.nc',
 's3://noaa-goes16/ABI-L1b-RadC/2000/001/12/OR_ABI-L1b-RadC-M3C01_G16_s20000011200000_e20000011200000_c20170951807462.nc',
 's3://noaa-goes

In [15]:
from dask.distributed import Client
client = Client()
client

/home/vboxuser/anaconda3/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43937 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:43937/status,
Dashboard: http://127.0.0.1:43937/status,Workers: 4
Total threads: 16,Total memory: 15.32 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36993,Workers: 4
Dashboard: http://127.0.0.1:43937/status,Total threads: 16
Started: Just now,Total memory: 15.32 GiB
Comm: tcp://127.0.0.1:37939,Total threads: 4
Dashboard: http://127.0.0.1:45807/status,Memory: 3.83 GiB
Nanny: tcp://127.0.0.1:35483,


In [16]:
def gen_json(u):
    so = dict(
        mode="rb", anon=True, default_fill_cache=False,
        default_cache_type="none"
    )
    with fsspec.open(u, **so) as inf:
        h5chunks = SingleHdf5ToZarr(inf, u, inline_threshold=300)
        with open(f"jsons/{u.split('/')[-1]}.json", 'wb') as outf:
           outf.write(ujson.dumps(h5chunks.translate()).encode())

In [ ]:
import dask
import pathlib
pathlib.Path('./jsons/').mkdir(exist_ok=True)
dask.compute(*[dask.delayed(gen_json)(u) for u in urls])